In [2]:
import time
import json
import pickle
import numpy as np
import pandas as pd
from classes import *
from datetime import datetime
import plotly.graph_objects as go

In [ ]:
with open("lpf_sensor_data.pkl", "rb") as f:
    sensor_data = pickle.load(f)

In [4]:
def plot_data_arr(sensor_data, mat, sensor, labels, skip=0, cut=0):
    title = f"Matrix {mat}, Sensor {sensor}"
    heaters = sensor_data[f"mat_{mat}"][sensor]
    fig = go.Figure()

    for i, data in heaters.items():
        fig.add_trace(go.Scatter(x=data["Time Since PowerOn"][skip:-cut],
                                 y=data["Filtered"][skip:-cut],
                                 mode="lines",
                                 showlegend=True,
                                 name=f"H.S. {i}"))
    
    for label in labels[f"mat_{mat}"]:
        fig.add_vrect(x0=label["start"],
                      x1=label["end"],
                      annotation_text=classes[label["label"]],
                      annotation_position="top left",
                      fillcolor=colors[label["label"]],
                      opacity=0.4,
                      line_width=0)
    fig.update_yaxes(title_text="Resistance (Ohms)",
                     type="log", tickformat=".0e")
    fig.update_xaxes(title_text="Time since power on (ms)", tickformat=".0e")
    fig.update_layout(height=400, width=700,
                      title_x=0.5,
                      font_family="Times New Roman",
                      title_font_family="Times New Roman",
                      title=dict(text=title, pad=dict(t=0, r=0, b=0, l=0)),
                      margin=dict(t=50, r=10, b=0, l=0))

    fig.update_yaxes(type="log")
    fig.show()

In [5]:
def get_df(matrix):
    with open(f"raw_data_raki/matrix_{matrix}/{matrix}.bmerawdata") as f:
        j = json.load(f)

    column_names = [el["name"] for el in j["rawDataBody"]["dataColumns"]]
    df = pd.DataFrame(j["rawDataBody"]["dataBlock"], columns=column_names)
    return df
    # return pd.read_csv(f"mat_{matrix}.csv");

In [6]:
def get_tspo_val(date_str, df):
    ts = time.mktime(datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S").timetuple())
    ts = int(ts)
    values = df[df["Real time clock"] == ts]["Time Since PowerOn"].values
    return values[0]

Matrix 0 -> alcohol; 1 -> control

In [1]:
df = get_df(0)
print(get_tspo_val("2025-12-12 11:10:00", df))
print(get_tspo_val("2025-12-12 15:55:00", df))
df = get_df(1)
print(get_tspo_val("2025-12-12 11:14:00", df))
print(get_tspo_val("2025-12-12 15:55:00", df))

NameError: name 'get_df' is not defined

In [20]:
label_datetimes = {
    "mat_0": [
        {"start": "2025-12-12 11:20:00",
         "end": "2025-12-12 11:40:00",
         "cls": ETH_100},
        {"start": "2025-12-12 12:10:00",
         "end": "2025-12-12 12:30:00",
         "cls": ETH_97_5},
        {"start": "2025-12-12 13:00:00",
         "end": "2025-12-12 13:20:00",
         "cls": ETH_95},
        {"start": "2025-12-12 13:50:00",
         "end": "2025-12-12 14:10:00",
         "cls": ETH_90},
        {"start": "2025-12-12 14:40:00",
         "end": "2025-12-12 15:00:00",
         "cls": ETH_80},
        {"start": "2025-12-12 15:30:00",
         "end": "2025-12-12 15:50:00",
         "cls": ETH_50}
    ],
    "mat_1": [
        {"start": "2025-12-12 11:20:00",
         "end": "2025-12-12 11:40:00",
         "cls": AIR},
        {"start": "2025-12-12 12:10:00",
         "end": "2025-12-12 12:30:00",
         "cls": AIR},
        {"start": "2025-12-12 13:00:00",
         "end": "2025-12-12 13:20:00",
         "cls": AIR},
        {"start": "2025-12-12 13:50:00",
         "end": "2025-12-12 14:10:00",
         "cls": AIR},
        {"start": "2025-12-12 14:40:00",
         "end": "2025-12-12 15:00:00",
         "cls": AIR},
        {"start": "2025-12-12 15:30:00",
         "end": "2025-12-12 15:50:00",
         "cls": AIR}
    ]
}

sensor_labels = {"mat_0": [], "mat_1": []}
for matrix in range(2):
    df = get_df(matrix)
    for label_time in label_datetimes[f"mat_{matrix}"]:
        start_ts = get_tspo_val(label_time["start"], df)
        end_ts = get_tspo_val(label_time["end"], df)
        cls = label_time["cls"]
        reg_target = regression_targets[cls] if cls in regression_targets else None
        
        sensor_labels[f"mat_{matrix}"].append({
            "start": start_ts,
            "end": end_ts,
            "label": cls,
            "target": reg_target
        })

In [19]:
plot_data_arr(sensor_data, 0, 7, sensor_labels, skip=10, cut=10)

In [30]:
plot_data_arr(sensor_data, 1, 0, sensor_labels, skip=10, cut=10)

In [31]:
with open("sensor_labels.pkl", "wb") as f:
    pickle.dump(sensor_labels, f)

In [32]:
def plot_matrices_data_arr(sensor_data, sensor, labels, skip=0, cut=0):
    title = f"Sensor {sensor}"
    mat = 0
    heaters = sensor_data[f"mat_{mat}"][sensor]
    fig = go.Figure()

    for i, data in heaters.items():
        fig.add_trace(go.Scatter(x=data["Time Since PowerOn"][skip:-cut],
                                 y=np.log(data["Filtered"][skip:-cut]),
                                 mode="lines",
                                 showlegend=True,
                                 name=f"0 H.S. {i}",
                                 line_color="#388eff"))
    
    for label in labels[f"mat_{mat}"]:
        fig.add_vrect(x0=label["start"],
                      x1=label["end"],
                      annotation_text=classes[label["label"]],
                      annotation_position="top left",
                      fillcolor=colors[label["label"]],
                      opacity=0.4,
                      line_width=0)
    mat = 1    
    heaters = sensor_data[f"mat_{mat}"][sensor]
    for i, data in heaters.items():
        fig.add_trace(go.Scatter(x=data["Time Since PowerOn"][skip:-cut],
                                 y=np.log(data["Filtered"][skip:-cut]),
                                 mode="lines",
                                 showlegend=True,
                                 name=f"1 H.S. {i}",
                                 line_color="#ff4538"))
        
    fig.update_yaxes(title_text="Resistance (Ohms)")
    fig.update_xaxes(title_text="Time since power on (ms)", tickformat=".0e")
    fig.update_layout(height=700, width=1200,
                      title_x=0.5,
                      font_family="Times New Roman",
                      title_font_family="Times New Roman",
                      title=dict(text=title, pad=dict(t=0, r=0, b=0, l=0)),
                      margin=dict(t=50, r=10, b=0, l=0))

    fig.show()

In [34]:
plot_matrices_data_arr(sensor_data, 1, sensor_labels, skip=10, cut=10)